#### ETL SOBRE STEAM_GAMES

In [5]:
import pandas as pd
import json
import ast
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
warnings.filterwarnings('ignore')

In [2]:
steam_games = 'datos_json/output_steam_games.json'

listado = []

with open(steam_games) as archivo:
    for linea in archivo.readlines():
        fila = json.loads(linea)
        listado.append(fila)

dfSteamGames = pd.DataFrame(listado)

#Reviso la cantidad de nulos por columna en el archivo
dfSteamGames.isnull().any()

publisher       True
genres          True
app_name        True
title           True
url             True
release_date    True
tags            True
reviews_url     True
specs           True
price           True
early_access    True
id              True
developer       True
dtype: bool

In [3]:
#Se limpian las filas donde todos los valores sean nulos y se reinicia el index 
games = dfSteamGames.dropna(how='all').reset_index(drop=True)

games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [4]:
games = games.drop_duplicates(subset='id')

In [5]:
'''  
OBSERVACIONES

id y price estan como objeto, debería transformarlos a int
release_date esta como objeto, podria transformarse a dato tipo date?
genres, tags y specs tienen datos anidados, deberia desanidarlos?


'''
games.dtypes

publisher       object
genres          object
app_name        object
title           object
url             object
release_date    object
tags            object
reviews_url     object
specs           object
price           object
early_access    object
id              object
developer       object
dtype: object

In [6]:
#En la API se va a utilizar la columna genre para algunas consultas, para facilitar su acceso se desanida la columna
games2 = games.explode('genres')

#Se reindexa el dataframe luego de desanidarse para su correcto acceso a la información
games2 = games2.reset_index(drop=True)

games2.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro


In [7]:
#almaceno los indices de los nulos en la columna id que no me permiten realizar la transformacion del tipo de dato
nulos = games2[games2['id'].isna()].index

#utilizo la variable nulos donde almacene el indice de nulos para ubicar las filas correspondientes
games2.iloc[nulos]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
189,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN


#### En base a lo observado:

- la fila 189 va a ser eliminada ya que un 80% de sus datos son nulos

In [8]:
games2.drop(index=189,inplace=True)

In [9]:
#se aplica la transformación del tipo en la columna id
games2['id'] = games2['id'].astype(int)

In [10]:
#Cambiamos la columna a tipo numero y los valores que sean letras que los ponga como nulos y los reemplazamos por 0
games2["price"] = pd.to_numeric(games2["price"], errors='coerce')

games2["price"] = games2["price"].fillna(0)

In [11]:
games2.dtypes

publisher        object
genres           object
app_name         object
title            object
url              object
release_date     object
tags             object
reviews_url      object
specs            object
price           float64
early_access     object
id                int32
developer        object
dtype: object

In [12]:
games2["release_date"] = pd.to_datetime(games2["release_date"],errors='coerce')

games2["year"] = games2["release_date"].dt.year

In [13]:
games2["year"].interpolate(method='ffill', inplace=True)

In [14]:
games2['year'] = games2['year'].astype(int)

In [15]:
games2.dropna(thresh= 4, inplace= True)

In [16]:
games2.isna().mean()

publisher       0.208371
genres          0.043858
app_name        0.000027
title           0.027394
url             0.000000
release_date    0.038726
tags            0.002459
reviews_url     0.000000
specs           0.012561
price           0.000000
early_access    0.000000
id              0.000000
developer       0.046477
year            0.000000
dtype: float64

In [17]:
#Se guardan las columnas a utilizar para las consultas en un nuevo dataframe
# REVISAR DE DONDE SAQUE QUE PRECIO IBA, revisar
steamGamesFinal = games2[['id','title','genres','price','year']]

steamGamesFinal

,id,title,genres,price,year
0,761140,Lost Summoner Kitty,Action,4.99,2018
1,761140,Lost Summoner Kitty,Casual,4.99,2018
2,761140,Lost Summoner Kitty,Indie,4.99,2018
3,761140,Lost Summoner Kitty,Simulation,4.99,2018
4,761140,Lost Summoner Kitty,Strategy,4.99,2018
...,...,...,...,...,...
74829,610660,Russian Roads,Racing,1.99,2018
74830,610660,Russian Roads,Simulation,1.99,2018
74831,658870,EXIT 2 - Directions,Casual,4.99,2017
74832,658870,EXIT 2 - Directions,Indie,4.99,2017


In [18]:
steamGamesFinal.to_csv('csv_limpios/steam_games.csv', index= False)

In [19]:
# Se convierte el DataFrame a una tabla de Arrow para posteriormente comprimirlo en parquet
table = pa.Table.from_pandas(steamGamesFinal)

# Especifica el nombre del archivo Parquet y comprime con snappy (puedes elegir otros algoritmos)
parquet_file = 'datos_parquet/steam_games.parquet'
pq.write_table(table, parquet_file, compression='snappy')